In [20]:
import pandas as pd

import matplotlib.pyplot as plt

import yfinance as yf

import mplfinance as mpf

from datetime import datetime

In [21]:
import mysql.connector
import redis
import json

# Establish a connection
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Vaibhav@123",
    database="stock_test"
)

# print(dir(conn))
# Create a cursor object to interact with the database
cursor = conn.cursor()

# Connect to Redis
redis_conn = redis.StrictRedis(host='localhost', port=6379, db=0)

In [22]:
import numpy as np
import datetime

# Define the time gap in minutes
time_gap = 20

# Function to calculate RSI (like TradingView)
def RSI(series, period=14):
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) # first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) # first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean()
    return 100 - 100 / (1 + rs)

# Function to calculate Stochastic RSI (like TradingView)
def StochRSI(series, period=14, smoothK=3, smoothD=3):
    rsi = RSI(series, period)
    rsi_min = rsi.rolling(window=period).min()
    rsi_max = rsi.rolling(window=period).max()
    stoch_rsi = 100 * (rsi - rsi_min) / (rsi_max - rsi_min)

    k = stoch_rsi.rolling(window=smoothK).mean()  # %K line
    d = k.rolling(window=smoothD).mean()  # %D line

    return k, d

def check_sell_signal(df):
    # Get the last two rows (current and previous) of %K and %D values
    last_row = df.iloc[-1]
    prev_row = df.iloc[-2]

    # Condition 1: K line crossing D line from above
    k_cross_d = prev_row['%K'] > prev_row['%D'] and last_row['%K'] < last_row['%D']
    # print(prev_row['%K'],prev_row['%D'],last_row['%K'],last_row['%D'])
    
    # Condition 2: Difference between the current and previous K values is more than 7
    k_difference = abs(last_row['%K'] - prev_row['%K']) > 2
    
    valid = prev_row['%K'] > 55 and last_row['%K'] < 95

    # Generate a sell signal if both conditions are met
    if k_cross_d and k_difference and valid:
        print("Generated the signal bro :)")
        return True
    else:
        return False
    
def check_buy_signal(df):
    # Get the last two rows (current and previous) of %K and %D values
    last_row = df.iloc[-1]
    prev_row = df.iloc[-2]

    # Condition 1: K line crossing D line from above
    k_cross_d = prev_row['%K'] < prev_row['%D'] and last_row['%K'] > last_row['%D']
    # print(prev_row['%K'],prev_row['%D'],last_row['%K'],last_row['%D'])
    
    # Condition 2: Difference between the current and previous K values is more than 7
    k_difference = abs(last_row['%K'] - prev_row['%K']) > 2
    
    valid = prev_row['%K'] > 10 and prev_row['%K'] < 96

    # Generate a sell signal if both conditions are met
    if k_cross_d and k_difference and valid:
        return True
    else:
        return False
    

def get_data(date, symbol):
    start_time = pd.Timestamp(f'{date} 00:00:00')
    end_time = pd.Timestamp(f'{date} 23:59:59')
    data = yf.download(symbol, start=start_time, end=end_time, interval='5m')
    data = data.rename(columns={'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'})

    # Replace missing volume data with zero
    data['volume'] = data['volume'].fillna(0)

    data.index = data.index.tz_localize(None)
    # Reset index to 'start_Time'
    data.index.name = 'start_Time'

    # data = data.drop('Adj Close')
    # print(type(data))
    data  = data.drop(columns=['Adj Close'])
    return data

# Function to simulate signal generation
def generate_signal(stock_id, signal_type):
    current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    if signal_type == 'sell':
        signal_data = {
            'stock_id': stock_id,
            'signal': 'S',
            'time': current_time
            
            
        }
        # Push the signal to Redis (could use a list or stream)
        redis_conn.rpush('stock_signals', json.dumps(signal_data))

    else:
        signal_data = {
            'stock_id': stock_id,
            'signal': 'B',
            'time': current_time
            
        }
        # Push the signal to Redis (could use a list or stream)
        redis_conn.rpush('stock_signals', json.dumps(signal_data))
        

def generate_signal_mysql(stock_id, signal_type):
    current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Check if the stock_id already exists in the signals table
    query = "SELECT time FROM stock_signals WHERE stock_name = %s"
    conn.commit()
    cursor.execute(query, (stock_id,))
    result = cursor.fetchone()

    if result:
        # Stock exists, check time difference
        last_signal_time = result[0]
        last_signal_time = datetime.datetime.strptime(str(last_signal_time), '%Y-%m-%d %H:%M:%S')
        time_diff = datetime.datetime.now() - last_signal_time

        # If the time difference is less than or equal to time_gap, don't update
        if time_diff.total_seconds() <= time_gap * 60:
            print("Signal not updated: Time gap too small")
            return
        else:
            # Update the signal in the table
            update_query = "UPDATE stock_signals SET stock_signal = %s, time = %s WHERE stock_name = %s"
            signal = 'S' if signal_type == "sell" else 'B'
            cursor.execute(update_query, (signal, current_time, stock_id))
            conn.commit()
            print("Signal updated successfully")
    else:
        # Stock doesn't exist, insert a new signal
        insert_query = "INSERT INTO stock_signals (stock_name, stock_signal,time) VALUES (%s, %s,%s)"
        signal = "S" if signal_type == "sell" else "B"
        cursor.execute(insert_query, (stock_id, signal,current_time))
        conn.commit()
        print("Signal inserted successfully")


def aggregate_to_5min_df(data):
    # Resampling the data to 15-minute intervals
    data_5min = data.resample('5T').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    })
    
    # Dropping any rows where the 'open' value is missing (incomplete 15 min intervals)
    data_5min = data_5min.dropna(subset=['open'])
    
    return data_5min

def get_dhan_data(token_code):
    data = dhan.intraday_minute_data(
        security_id=token_code,
        exchange_segment='NSE_EQ',
        instrument_type='EQUITY'
    )
    # print(data)
    data = data['data']
    # print(da)
    df1 = pd.DataFrame(data)
    
    # print(df1)
    # print(data)
    df1['start_Time'] = df1['start_Time'].apply(dhan.convert_to_date_time)
    df1.set_index('start_Time', inplace=True)
    df1=aggregate_to_5min_df(df1)
    return df1
    # print(df.head(152))
    # data = aggregate_to_15min(data)
    # print("----------------------------------------------------------")
    # df = pd.DataFrame(data)
    # df['start_Time'] = df['start_Time'].apply(dhan.convert_to_date_time)
    # # df['start_Time'] = df['start_Time'].apply(dhan.convert_to_date_time)
    


def aggregate_to_15min_df(data):
    # Resampling the data to 15-minute intervals
    data_15min = data.resample('15T').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    })
    
    # Dropping any rows where the 'open' value is missing (incomplete 15 min intervals)
    data_15min = data_15min.dropna(subset=['open'])
    
    return data_15min




# def check_sell_signal(df):
#     # Get the last two rows (current and previous) of %K and %D values
#     last_row = df.iloc[-1]
#     prev_row = df.iloc[-2]

#     # Condition 1: K line crossing D line from above
#     k_cross_d = prev_row['%K'] > prev_row['%D'] and last_row['%K'] < last_row['%D']
#     # print(prev_row['%K'],prev_row['%D'],last_row['%K'],last_row['%D'])
    
#     # Condition 2: Difference between the current and previous K values is more than 7
#     k_difference = abs(last_row['%K'] - prev_row['%K']) > 2
    
#     valid = prev_row['%K'] > 55 and last_row['%K'] < 95

#     # Generate a sell signal if both conditions are met
#     if k_cross_d and k_difference and valid:
#         return True
#     else:
#         return False


In [4]:
import time
import datetime



query_for_yf_names = "select yf_name from stocks"

cursor.execute(query_for_yf_names)


result = cursor.fetchall()

dates = [ "2024-10-14", "2024-10-15" , "2024-10-16" , "2024-10-17" , "2024-10-18" , "2024-10-21" ,"2024-10-22", "2024-10-23" , "2024-10-24"  ]


yf_name_list = [row[0] for row in result]

while True:
    current_time = datetime.datetime.now().time()
    if  (current_time >= datetime.time(9, 0) and current_time <= datetime.time(12,0) ) or (current_time >= datetime.time(13, 30) and current_time <= datetime.time(15,0)):

        for stock_name in yf_name_list:
            
            combined_data = get_data(dates[0],stock_name)
            # print(type(combined_data))
            for date in dates:
                data = get_data(date,stock_name)
                # print(type(data))
                combined_data = pd.concat([combined_data,data])
            
            query = "select stock_number from stocks where yf_name=%s"
            cursor.execute(query,(stock_name,))
            result = cursor.fetchall()
            token = result[0][0]
            dhan_security_code = str(token)
            dhan_data = get_dhan_data(dhan_security_code)
            # query_for_yf_names = "select stock_number from stocks where yf_name=%s"
            # cursor.execute(query_for_yf_names,("ABC",))
            
            
            combined_data = pd.concat([combined_data,dhan_data])
            combined_data = aggregate_to_15min_df(combined_data)
            combined_data = combined_data.reset_index(drop=True)
            
            data_new = combined_data
            data_new['%K'], data_new['%D'] = StochRSI(data_new['close'])
            # print(check_sell_signal(combined_data)
            stock_name_here = stock_name[:-3]
            if check_sell_signal(combined_data)==True:
                generate_signal_mysql(stock_name_here, 'sell')
            elif check_buy_signal(combined_data)==True:
                generate_signal_mysql(stock_name_here, 'buy')
        
        time.sleep(20)
            
    else:
        print("Markets are closed")
        break
        
    # print(combined_data)


# print(combined_data)
        


# print(yf_name_list)

Markets are closed


In [14]:
df = get_data("2024-10-14" , "DLF.NS")
print(df)
# print("hi")/\

[*********************100%%**********************]  1 of 1 completed

                           open        high         low       close  volume
start_Time                                                                 
2024-10-14 09:15:00  852.049988  852.049988  846.150024  848.299988       0
2024-10-14 09:20:00  848.000000  848.099976  845.650024  846.500000   43142
2024-10-14 09:25:00  846.500000  846.500000  844.150024  844.400024   23929
2024-10-14 09:30:00  844.700012  848.750000  844.000000  848.299988   60011
2024-10-14 09:35:00  848.549988  850.650024  847.599976  849.799988   53554
...                         ...         ...         ...         ...     ...
2024-10-14 15:05:00  863.750000  864.200012  862.200012  862.799988   90577
2024-10-14 15:10:00  862.950012  863.000000  862.200012  862.200012   40095
2024-10-14 15:15:00  862.250000  863.250000  861.549988  863.099976   75561
2024-10-14 15:20:00  862.849976  863.849976  862.549988  862.799988   63441
2024-10-14 15:25:00  862.750000  863.250000  861.799988  861.799988   46799

[75 rows x 

In [36]:
def get_stock_data_from_mysql(date: str, ticker: str) -> pd.DataFrame:
    
    
    # Query the data
    query = """
        SELECT timestamp AS start_Time, open, high, low, close, volume
        FROM stock_data_test
        WHERE date = %s AND ticker = %s
        ORDER BY timestamp
    """
    # Use pandas to execute the query and load the data directly into a DataFrame
    df = pd.read_sql(query, conn, params=(date, ticker))
    
    
    # Set 'start_Time' as the index
    df.set_index('start_Time', inplace=True)
    
    return df



In [38]:

# df = get_data("2024-10-14", "DLF.NS")

# ticker = "DLF.NS"
# date = "2024-10-14"


def update_data_mysql(date,ticker,df):
    # Iterate through each row in the DataFrame and insert/update in MySQL
    for timestamp, row in df.iterrows():
        timestamp = timestamp.to_pydatetime()

        # Convert all row values to native Python types
        open_val = float(row['open'])
        high_val = float(row['high'])
        low_val = float(row['low'])
        close_val = float(row['close'])
        volume_val = int(row['volume'])  # Assuming volume is an integer
        percent_k = float(row['%K'])
        percent_d = float(row["%D"])

        cursor.execute("""
            INSERT INTO stock_data_test_2 (ticker, date, timestamp, open, high, low, close, volume,percent_k,percent_d)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s,%s)
            ON DUPLICATE KEY UPDATE
                open = VALUES(open),
                high = VALUES(high),
                low = VALUES(low),
                close = VALUES(close),
                volume = VALUES(volume),
                percent_k = VALUES(percent_k),
                percent_d = VALUES(percent_d)
        """, (ticker, date, timestamp, open_val, high_val, low_val, close_val, volume_val,percent_k,percent_d))

    conn.commit()

In [42]:
query_for_yf_names = "select yf_name from stocks"

cursor.execute(query_for_yf_names)


result = cursor.fetchall()

dates = [ "2024-10-18"]

yf_name_list = [row[0] for row in result]

for stock_name in yf_name_list:
    combined_data = get_data(dates[0],stock_name)
    # print(type(combined_data))
    for date in dates:
        data = get_data(date,stock_name)
        update_data_mysql(date,stock_name,data)
        
        # print(type(data))
        

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [39]:
data = get_data(  "2024-10-14","DLF.NS" )
print(data)
data['%K'], data['%D'] = StochRSI(data['close'])
print(data)

update_data_mysql("2024-10-14","DLF.NS" , data)

[*********************100%%**********************]  1 of 1 completed

                           open        high         low       close  volume
start_Time                                                                 
2024-10-14 09:15:00  852.049988  852.049988  846.150024  848.299988       0
2024-10-14 09:20:00  848.000000  848.099976  845.650024  846.500000   43142
2024-10-14 09:25:00  846.500000  846.500000  844.150024  844.400024   23929
2024-10-14 09:30:00  844.700012  848.750000  844.000000  848.299988   60011
2024-10-14 09:35:00  848.549988  850.650024  847.599976  849.799988   53554
...                         ...         ...         ...         ...     ...
2024-10-14 15:05:00  863.750000  864.200012  862.200012  862.799988   90577
2024-10-14 15:10:00  862.950012  863.000000  862.200012  862.200012   40095
2024-10-14 15:15:00  862.250000  863.250000  861.549988  863.099976   75561
2024-10-14 15:20:00  862.849976  863.849976  862.549988  862.799988   63441
2024-10-14 15:25:00  862.750000  863.250000  861.799988  861.799988   46799

[75 rows x 

In [13]:
import time


current_time = datetime.now()
time.sleep(1)
end_time = datetime.now()
print(end_time)
# time1 = datetime.timedelta()
diff = end_time-current_time
print(f"the time difference is {diff.seconds}")

2024-10-27 18:03:50.046369
the time difference is 1


In [15]:
insert_trade_query = "INSERT INTO all_trades (time,stock_name, stock_action,target,stop_loss,result,entry_price,exit_price, change_percentage, entry_ltp) VALUES (%s, %s,%s,%s,%s,%s,%s,%s,%s,%s)"
current_time = datetime.now()
stock_name = "xyz"
stock_action = "B"
target = 100.09
stop_loss = 99.9
result = "profit"
entry_price = 100
exit_price = 100.1
change_percentage = 0.1
entry_ltp = 100
values = (current_time, stock_name , stock_action , target , stop_loss , result, entry_price , exit_price , change_percentage , entry_ltp)
cursor.execute(insert_trade_query, values)
conn.commit()


In [19]:
change_percentage = 10


result = "profit" if change_percentage>=0 else "loss"
print(result)

profit
